<a href="https://colab.research.google.com/github/thyarles/unb-fmc-nlp/blob/main/aula_1/notes_lets_build_the_gpt_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's build the GPT Tokenizer
Notes from the video https://www.youtube.com/watch?v=zduSFxRajkE.

* Most of the problem we see on the LLM are from Tokenizers (like do simple math wrong).
* The unicode has three types, UTF-8, UTF-16, and UTF-32. The UTF-8 is the standard because it the only one that has variable length. For latin characters, the UTF-16 add zero word on every letter, and UTF-32 add two zero words.
* We can't use the Unicode to tokenizer because it has a huge code space (about 150 thousand).  

In [ ]:
# To check the UTF-8 value
[ord(x) for x in "Charles."]

[67, 104, 97, 114, 108, 101, 115, 46]

In [ ]:
# To check the UTFs 8, 16 and 32.
print("%s\n%s\n%s" %
(
  list("Charles.".encode("utf-8")),
  list("Charles.".encode("utf-16")),
  list("Charles.".encode("utf-32")))
)

[67, 104, 97, 114, 108, 101, 115, 46]
[255, 254, 67, 0, 104, 0, 97, 0, 114, 0, 108, 0, 101, 0, 115, 0, 46, 0]
[255, 254, 0, 0, 67, 0, 0, 0, 104, 0, 0, 0, 97, 0, 0, 0, 114, 0, 0, 0, 108, 0, 0, 0, 101, 0, 0, 0, 115, 0, 0, 0, 46, 0, 0, 0]


## Byte pair encoding

### Encoding

In [7]:
# Example: aaabdaaabac (vocabulary size = eleven, four tokens)
#          -> find the pair that occurs more frequently and replace it with a single token
#          Z = aa, Y = ab, X = zy -> XdXac (vocabulary size = seven, five tokens)

# Fancy chars consume more bytes, that's why the code points is less than tokens
text = """
  Alan Turing foi um matemático e criptógrafo inglês considerado atualmente como o
  pai da computação, uma vez que, por meio de suas ideias, foi possível desenvolver
  o que chamamos hoje de computador. Turing também ficou muito conhecido como um dos
  responsáveis por decifrar o código utilizado pelas comunicações nazistas durante
  a Segunda Guerra Mundial.

  Por meio do seu trabalho, foi desenvolvida uma máquina conhecida como “bomba
  eletromecânica” (The Bombe, em inglês), que decifrou o código da máquina Enigma
  utilizado pelos alemães, e permitiu que os Aliados tivessem acesso a informações
  privilegiadas ao longo da guerra. Turing morreu em 1954, provavelmente tendo
  cometido suicídio.
"""
tokens = text.encode("utf-8")     # raw bytes
tokens = list(map(int, tokens))   # integers from 0 to 255
print("The text has %d code points and %d tokens." % (len(text), len(tokens)))

The text has 717 code points and 741 tokens.


In [3]:
# Let's find the most frequent value

# Using the pythonic way
def get_stats_pythonic(ids):
  counts = {}
  for pair in zip(ids, ids[1:]):
    counts[pair] = counts.get(pair, 0) + 1
  return counts

# Using human way
def get_stats_human(ids):
    counts = {}
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i + 1])
        if pair in counts:
            counts[pair] += 1
        else:
            counts[pair] = 1
    return counts

stats = get_stats_pythonic(tokens)
print(sorted(((v, k) for k,v in stats.items()), reverse=True))

[(24, (111, 32)), (15, (97, 32)), (14, (32, 99)), (13, (115, 32)), (12, (101, 32)), (12, (99, 111)), (12, (32, 100)), (10, (111, 109)), (10, (100, 111)), (10, (32, 32)), (10, (10, 32)), (9, (32, 112)), (8, (105, 110)), (8, (101, 115)), (8, (100, 101)), (8, (44, 32)), (8, (32, 10)), (7, (114, 97)), (7, (100, 97)), (7, (32, 109)), (6, (118, 101)), (6, (117, 101)), (6, (116, 105)), (6, (113, 117)), (6, (111, 115)), (6, (111, 114)), (6, (110, 103)), (6, (109, 101)), (6, (109, 97)), (6, (109, 32)), (6, (105, 100)), (6, (97, 100)), (6, (32, 117)), (6, (32, 97)), (5, (117, 32)), (5, (116, 101)), (5, (114, 105)), (5, (114, 32)), (5, (111, 110)), (5, (109, 111)), (5, (105, 99)), (5, (102, 111)), (5, (101, 114)), (5, (101, 110)), (5, (101, 109)), (5, (101, 108)), (5, (101, 99)), (5, (97, 115)), (5, (32, 111)), (5, (32, 101)), (4, (195, 161)), (4, (117, 116)), (4, (117, 114)), (4, (117, 109)), (4, (117, 105)), (4, (116, 97)), (4, (115, 101)), (4, (112, 111)), (4, (109, 195)), (4, (105, 97)), (4, 

In [4]:
# Let's see what are the most printed values
chr(111), chr(32) # this is the opposite of ord(x)

('o', ' ')

In [5]:
# Let's create news tokens starting from 256
top_pair = max(stats, key=stats.get)

def merge(ids, pair, idx):
  # replace ids of the pair with idx
  new_ids = []
  i = 0
  while i < len(ids):
    # if not on the last position and finds, replace it
    if i < len(ids) -1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)
      i += 2
    else:
      new_ids.append(ids[i])
      i += 1
  return new_ids

# To check
# print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))
new_tokens = merge(tokens, top_pair, 256)
# The result should change from 741 to 717 as we had 24 pairs of ('o', ' ')
print("The text has %d code points and %d tokens." % (len(text), len(new_tokens)))

The text has 717 code points and 717 tokens.


In [6]:
# The more changes you make, bigger will be your vocabulary and shorter will be
# your text. You need to find the best balance.

vocabulary_size = 276 # desired vocab size
number_merges = vocabulary_size - 256 # minus what we have already
ids = list(tokens) # let's keep the original list intact (use list to copy)

merges = {}
for i in range(number_merges):
  stats = get_stats_pythonic(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"Merging in a token {idx} the pair {pair}...")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

Merging in a token 256 the pair (111, 32)...
Merging in a token 257 the pair (97, 32)...
Merging in a token 258 the pair (115, 32)...
Merging in a token 259 the pair (101, 32)...
Merging in a token 260 the pair (99, 111)...
Merging in a token 261 the pair (10, 32)...
Merging in a token 262 the pair (261, 32)...
Merging in a token 263 the pair (105, 110)...
Merging in a token 264 the pair (44, 32)...
Merging in a token 265 the pair (100, 256)...
Merging in a token 266 the pair (260, 109)...
Merging in a token 267 the pair (109, 32)...
Merging in a token 268 the pair (116, 105)...
Merging in a token 269 the pair (114, 97)...
Merging in a token 270 the pair (100, 101)...
Merging in a token 271 the pair (100, 257)...
Merging in a token 272 the pair (118, 101)...
Merging in a token 273 the pair (113, 117)...
Merging in a token 274 the pair (111, 114)...
Merging in a token 275 the pair (263, 103)...


In [37]:
# Let's see the compression we got
print(f'Tokens before merge: {len(tokens)}')
print(f'Tokens after merge: {len(ids)}')
print(f'Ratio: {len(tokens)/len(ids):.2f}x')

Tokens before merge: 741
Tokens after merge: 563
Ratio: 1.32x


In [8]:
def encode(text):
  # Given the string, retir the ids (list of integers, aka tokens)
  tokens = list(text.encode('utf-8'))
  while len(tokens) > 1:
    stats = get_stats_pythonic(tokens)
    # float('inf') is a fallback, a big number if something goes wrong on the index
    pair = min(stats, key=lambda p: merges.get(p, float('inf')))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

### Decoding

In [17]:
# Populate the vocab with map from 0 to 255
vocabulary = {idx: bytes([idx]) for idx in range(256)}

# Now let's get the remaining items from merges
for (p0, p1), idx in merges.items():
  # vocabulary is just a byte object, so the plus is a concatenation
  vocabulary[idx] = vocabulary[p0] + vocabulary[p1]

def decode(idx):
  # Given the ids (list of integers), find the string related
  text_bytes = b''.join(vocabulary[idx] for idx in ids)
  # The errors can be strict, ignore, replace, backslashreplace or surrogatescape
  return text_bytes.decode(encoding='utf-8', errors='replace')

# Python code

In [67]:
# BPE (Byte-Pair Encoding)

# Quanto maiores forem as modificações, maior será o vocabulário e menor será
# o texto. Precisamos encontrar o equilíbrio.

# __init__      : chama o criador do vocabulario e um dicionário vazio
# _build_vocab  : cria o vocabulário
# _get_stats    : conta a ocorrência dos pares em determinada string
# _merge        : troca as ocorrências de maior frequência por novo vocabulário
# train         : procura os caracteres frequentes e une, criando novo vocabulário
# encoding      : converte texto em ids
# decoding      : converte ids em texto
# print_subwords: imprime a representação criada no treinamento

class MyBPE():

    # Inicializa dicionário para merges e vocabulário, tornando-os disponíveis
    # durante toda existência da classe
    def __init__(self):
        self.merges = {}
        self.vocab = self._build_vocab()

    # Cria o vocabulário, é chamada imediatamente após a instância da classe
    def _build_vocab(self):
        # Cria o vocabulário inicial, com valores de 0 a 255
        vocab = {idx: bytes([idx]) for idx in range(256)}
        # Estende o vocabulário com base nos valores que tiveram merge
        # Observe que a soma é uma concatenação, pois vocab é byte
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        return vocab


    # Conta as ocorrências de cada par em determinado ids
    def _get_stats(self, ids):
        counts = {}
        for pair in zip(ids, ids[1:]):
            # Incrementa a cada par
            counts[pair] = counts.get(pair, 0) + 1
        return counts
    # Outra forma de fazer (versão mais humana e menos pytônica)
      # def _get_stats(ids):
      # counts = {}
      # for i in range(len(ids) - 1):
      #     pair = (ids[i], ids[i + 1])
      #     if pair in counts:
      #         counts[pair] += 1
      #     else:
      #         counts[pair] = 1
      # return counts


    # Troca determinado par por um novo vocabulário
    def _merge(self, ids, pair, idx):
        newids = []
        i = 0
        while i < len(ids):
            # Verifica o par na sequência e pula se houver merge (+2)
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            # Não teve merge, mantenha o valor e vai para o próximo
            else:
                newids.append(ids[i])
                i += 1
        return newids


    # Treina o modelo adicionando cada par unido no dicionário
    def train(self, text, vocab_size):
        # Se o vocabulário for maior que 255, sinalize o erro
        assert vocab_size >= 256
        # O vocabulário já tem 256, calcule a diferença
        num_merges = vocab_size - 256
        # Converta o texto de entrada para bytes, usando unicode UTF-8
        text_bytes = text.encode("utf-8")
        # Cada byte tem que ser um elemento independente
        ids = list(text_bytes)
        # Dicionário para salvar o processamento local
        merges = {}
        # Vocabulário com os valores de 0 a 255
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for i in range(num_merges):
            # Pega a frequênica dos pares
            stats = self._get_stats(ids)
            # Procura pelo de maior frequência
            pair = max(stats, key=stats.get)
            # Incrementa identificador do vocabulário
            idx = 256 + i
            # Faz a união do par no identificador idx
            ids = self._merge(ids, pair, idx)
            # Registra como uma união para operações de [de]codificação
            merges[pair] = idx
            # Atualiza vocabulário
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
        # Salva os merges na instância
        self.merges = merges
        # Salva o vocabulário na instância
        self.vocab = vocab


    # Decodifica determinados ids em texto
    def decode(self, ids):
        # Cria a cadeia de bytes
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        # Decodifica usando UTF-8: se tiver erro de conversão, troque
        text = text_bytes.decode("utf-8", errors="replace")
        return text


    # Codifica determinado texto em ids
    def encode(self, text):
        # Converte o texto para byte usando UTF-8
        text_bytes = text.encode("utf-8")
        # Converte em lista para que cada byte seja independente
        ids = list(text_bytes)
        # A lista tem que ter pelo menos 2 elementos, do contrário
        # não faz qualquer sentido o merge
        while len(ids) >= 2:
            stats = self._get_stats(ids)
            # float('inf') é um fallback, um número gigante que garantirá um
            # valor mínimo caso tenha algo errado no index
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                # Se chegou aqui, não há nada mais para unir, saia
                break
            # Recupere o idx da instância
            idx = self.merges[pair]
            # Recuere o ids da instância e faça a união do par
            ids = self._merge(ids, pair, idx)
        return ids


    # Imprime a lista de subpalavras criadas no treinamento
    def print_subwords(self):
      merges = sorted(self.merges)
      # Como o vocabulário base tem 256 itens, vamos começar do 257
      i = 257
      for tokens in merges:
        print(f'Vocabulary {i} with tokens {tokens} decodes to "{self.decode(tokens)}"')
        i += 1


In [68]:
teste = MyBPE()
teste.train("hello world", 260)

In [69]:
teste.print_subwords()

Vocabulary 257 with tokens (104, 101) decodes to "he"
Vocabulary 258 with tokens (256, 108) decodes to "hel"
Vocabulary 259 with tokens (257, 108) decodes to "hell"
Vocabulary 260 with tokens (258, 111) decodes to "hello"
